Lectura de archivo en excel con todas sus hojas

In [50]:
# Importar las librerías necesarias
import pandas as pd

# Especificar la ruta del archivo en "files"
file_path = '/home/elicoubuntu/Desktop/Industrial_Insigths/Prognosis/subestacion _canaima _1000kva.xlsx'

# Cargar el archivo de Excel
excel_data = pd.ExcelFile(file_path)

# Ver las hojas del archivo de Excel
excel_data.sheet_names


['tension',
 'corriente',
 'energia',
 'Factor P',
 'Flickers (Pst)',
 'Flickers (Plt)',
 'Frecuencia',
 'potencia A',
 'Potencia Aparente',
 'potencia R I',
 'Distorsion armonica',
 'Armonicos  IL1',
 'Armonicos  IL2',
 'Armonicos  IL3',
 'Armonicos  VL1',
 'Armonicos  VL2',
 'Armonicos  VL3']

Consolidacion de datos es un solo data set por marca de tiempo

In [51]:
# Función para consolidar las hojas del Excel basadas en 'Fecha' y 'Tiempo'
def consolidate_sheets(excel_data, sheet_names):
    consolidated_df = None
    
    for sheet in sheet_names:
        sheet_df = excel_data.parse(sheet)
        
        # Combinar Fecha y Tiempo en una sola columna timestamp
        if 'Fecha' in sheet_df.columns and 'Tiempo' in sheet_df.columns:
            # Convertir 'Fecha' a string si es necesario
            if pd.api.types.is_datetime64_any_dtype(sheet_df['Fecha']):
                sheet_df['Fecha'] = sheet_df['Fecha'].dt.strftime('%Y-%m-%d')
            
            # Convertir 'Tiempo' a string si es de tipo datetime.time
            sheet_df['Tiempo'] = sheet_df['Tiempo'].apply(lambda x: x.strftime('%H:%M:%S') if isinstance(x, pd._libs.tslibs.timestamps.Timestamp) or isinstance(x, pd._libs.tslibs.nattype.NaTType) else x.strftime('%H:%M:%S'))
            
            # Crear la columna timestamp
            sheet_df['timestamp'] = pd.to_datetime(sheet_df['Fecha'] + ' ' + sheet_df['Tiempo'])
            sheet_df = sheet_df.drop(columns=['Fecha', 'Tiempo'])
        
        # Unir todas las hojas
        if consolidated_df is None:
            consolidated_df = sheet_df
        else:
            consolidated_df = pd.merge(consolidated_df, sheet_df, on='timestamp', how='outer')

    return consolidated_df

# Consolidar las hojas
sheet_names = excel_data.sheet_names
consolidated_data = consolidate_sheets(excel_data, sheet_names)

# Ver una muestra de los datos consolidados
print(consolidated_data.head())

# Guardar el DataFrame consolidado en un archivo Excel
output_excel_path = '/home/elicoubuntu/Desktop/Industrial_Insigths/Prognosis/consolidated_data.xlsx'
consolidated_data.to_excel(output_excel_path, index=False)

print(f"Datos consolidados guardados en: {output_excel_path}")


   Tensión MIN: L1 (V)  Tensión: L1 (V)  Tensión MAX: L1 (V)  \
0               264.21           264.42               264.73   
1               264.37           264.73               264.93   
2               264.57           264.78               265.02   
3               264.54           264.77               265.00   
4               264.50           264.85               265.07   

   Tensión MIN: L2 (V)  Tensión: L2 (V)  Tensión MAX: L2 (V)  \
0               268.12           268.32               268.46   
1               268.42           268.59               268.80   
2               268.52           268.65               268.85   
3               268.29           268.60               268.88   
4               268.41           268.65               268.85   

   Tensión MIN: L3 (V)  Tensión: L3 (V)  Tensión MAX: L3 (V)  \
0               266.51           266.73               266.92   
1               266.69           266.98               267.20   
2               266.87           267.0

In [89]:
consolidated_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7141 entries, 0 to 7140
Columns: 382 entries, Tensión MIN: L1 (V) to Armónicos VL3: Armónico 50 (%VL3)
dtypes: datetime64[ns](1), float64(111), int64(267), object(3)
memory usage: 20.8+ MB


Validacion de datos

In [53]:
# Validación de los datos consolidados

# 1. Verificar si hay valores faltantes
missing_data = consolidated_data.isnull().sum()
print("Valores faltantes por columna:")
print(missing_data[missing_data > 0])

# 2. Verificar si hay registros duplicados basados en 'timestamp'
duplicates = consolidated_data.duplicated(subset='timestamp').sum()
print(f"\nNúmero de registros duplicados: {duplicates}")

# 3. Verificar los tipos de datos de las columnas
print("\nTipos de datos por columna:")
print(consolidated_data.dtypes)

# 4. Generar estadísticas descriptivas de las columnas numéricas
print("\nEstadísticas descriptivas:")
print(consolidated_data.describe())


Valores faltantes por columna:
Series([], dtype: int64)

Número de registros duplicados: 0

Tipos de datos por columna:
Tensión MIN: L1 (V)                  float64
Tensión: L1 (V)                      float64
Tensión MAX: L1 (V)                  float64
Tensión MIN: L2 (V)                  float64
Tensión: L2 (V)                      float64
                                      ...   
Armónicos VL3: Armónico 46 (%VL3)      int64
Armónicos VL3: Armónico 47 (%VL3)      int64
Armónicos VL3: Armónico 48 (%VL3)      int64
Armónicos VL3: Armónico 49 (%VL3)      int64
Armónicos VL3: Armónico 50 (%VL3)      int64
Length: 382, dtype: object

Estadísticas descriptivas:
       Tensión MIN: L1 (V)  Tensión: L1 (V)  Tensión MAX: L1 (V)  \
count          7141.000000      7141.000000          7141.000000   
mean            264.445552       264.734348           265.000815   
min             257.010000       258.960000           259.320000   
25%             262.740000       263.040000           263.

In [54]:
import pandas as pd

# Carga el archivo de datos
file_path = '/home/elicoubuntu/Desktop/Industrial_Insigths/Prognosis/consolidated_data.xlsx'
data = pd.read_excel(file_path)

# Muestra todas las columnas del archivo para verificación
print(data.columns.tolist())


['Tensión MIN: L1 (V)', 'Tensión: L1 (V)', 'Tensión MAX: L1 (V)', 'Tensión MIN: L2 (V)', 'Tensión: L2 (V)', 'Tensión MAX: L2 (V)', 'Tensión MIN: L3 (V)', 'Tensión: L3 (V)', 'Tensión MAX: L3 (V)', 'Tensión MIN: L1 - L2 (V)', 'Tensión: L1 - L2 (V)', 'Tensión MAX: L1 - L2 (V)', 'Tensión MIN: L2 - L3 (V)', 'Tensión: L2 - L3 (V)', 'Tensión MAX: L2 - L3 (V)', 'Tensión MIN: L3 - L1 (V)', 'Tensión: L3 - L1 (V)', 'Tensión MAX: L3 - L1 (V)', 'Tensión MIN: Trifásica (f-f) ~ (V)', 'Tensión: Trifásica (f-f) ~ (V)', 'Tensión MAX: Trifásica (f-f) ~ (V)', 'timestamp', 'Corriente MIN: L1 (A)', 'Corriente: L1 (A)', 'Corriente MAX: L1 (A)', 'Corriente MIN: L2 (A)', 'Corriente: L2 (A)', 'Corriente MAX: L2 (A)', 'Corriente MIN: L3 (A)', 'Corriente: L3 (A)', 'Corriente MAX: L3 (A)', 'Corriente MIN: In (A)', 'Corriente: In (A)', 'Corriente MAX: In (A)', 'Energías (Tarifa): Activa + (Total) (kWh)', 'Energías (Tarifa): Inductiva + (Total) (kvarh)', 'Energías (Tarifa): Aparente + (Total) (VAh)', 'Factor de pote

Basados en los criterios tecnicos de expertos, las variables eléctricas clave que pueden ser las más representativas:

Tensiones de línea y fase (Tensión L1, L2, L3, y sus mínimas y máximas):

Razón: La estabilidad y calidad del suministro eléctrico depende directamente de la tensión. Cualquier desviación significativa en las tensiones, especialmente si se sale de los rangos permitidos, puede indicar problemas en el sistema, como sobrecargas, desequilibrio de fases, o incluso falla en transformadores.
Importancia: Las tensiones fuera de los rangos normales pueden generar sobrecalentamiento o desbalance que impacta en la eficiencia y puede ser indicativo de fallas inminentes.

Corrientes de fase (Corriente L1, L2, L3):

Razón: Las corrientes reflejan directamente el consumo de energía y las cargas conectadas al sistema. Un incremento o fluctuación en las corrientes puede estar relacionado con sobrecargas, cortocircuitos o fallas en los equipos de la subestación.
Importancia: Es esencial monitorear estas corrientes para detectar cualquier sobrecarga que podría derivar en un fallo.

Potencias (Aparente, Activa, Reactiva):

Razón: La potencia activa (kW) es la que realmente realiza trabajo útil, mientras que la potencia reactiva (kVAR) es necesaria para mantener los campos eléctricos y magnéticos en los equipos. Un desbalance entre estos dos tipos de potencias puede indicar problemas en la eficiencia energética o fallas en los equipos que generen exceso de consumo de energía reactiva.
Importancia: El exceso de potencia reactiva puede sobrecargar los transformadores y sistemas de distribución, mientras que la potencia activa refleja el trabajo real. El monitoreo de la potencia total (aparente) permite observar posibles sobrecargas.

Factor de potencia (FP):

Razón: Un factor de potencia bajo significa que se está consumiendo una gran cantidad de potencia reactiva en comparación con la potencia activa, lo cual es ineficiente y puede causar sobrecargas en el sistema.
Importancia: Es un indicador importante de eficiencia. Valores bajos deben corregirse y pueden estar asociados a fallas en los equipos de corrección del factor de potencia, como los condensadores.

Distorsión armónica (THD) en tensión y corriente:

Razón: La distorsión armónica indica la presencia de armónicos (frecuencias no deseadas) en la señal de tensión o corriente, que pueden provenir de equipos electrónicos no lineales o cargas como motores de velocidad variable o rectificadores. Estos armónicos causan pérdidas adicionales y posibles fallas en los transformadores y otros equipos.
Importancia: Niveles elevados de distorsión armónica pueden generar un sobrecalentamiento en los transformadores, aumentar las pérdidas y reducir la vida útil del equipo.

Temperatura de los transformadores y equipos:

Razón: Los sistemas eléctricos se ven afectados por el calor, especialmente los transformadores, que pueden sobrecalentarse bajo carga. Monitorear la temperatura ayuda a prevenir fallas térmicas.
Importancia: Las temperaturas fuera de rango pueden indicar problemas como sobrecarga, aislamiento defectuoso o mal funcionamiento de los sistemas de enfriamiento.

Frecuencia del sistema:

Razón: La frecuencia debe mantenerse constante en el sistema de distribución (normalmente alrededor de 50 o 60 Hz, dependiendo de la región). Cambios significativos en la frecuencia pueden ser una indicación de desbalance entre generación y consumo, o problemas serios en el sistema.
Importancia: Una frecuencia fuera del rango normal puede causar problemas en equipos sensibles y reflejar fallas en el sistema de control de la subestación.

Variables secundarias que podrían no ser prioritarias:

Armónicos de órdenes superiores (> 15): Si bien los armónicos de orden bajo pueden tener mayor impacto, los de órdenes más altos (20, 30, 40, etc.) pueden no ser tan críticos para la predicción de fallas en subestaciones.

Variables de eventos específicos: Dependiendo del sistema, algunos eventos puntuales o picos pueden ser menos importantes si ocurren con poca frecuencia o no afectan el consumo energético general.

Conclusión:
Las variables que reflejan el comportamiento general del sistema eléctrico, como tensiones, corrientes, potencias, distorsión armónica y factor de potencia, son las más importantes para la predicción de fallas. Variables relacionadas con condiciones específicas, como armónicos de orden alto o eventos puntuales, podrían ser consideradas secundarias y podrían eliminarse o reducirse en el análisis.

In [90]:
import pandas as pd

# Ruta del archivo original
file_path = '/home/elicoubuntu/Desktop/Industrial_Insigths/Prognosis/consolidated_data.xlsx'
data = pd.read_excel(file_path)

# Lista de columnas a conservar, ajustada según los nombres exactos en el archivo
columns_to_keep = [
    "Tensión: L1 (V)", "Tensión: L2 (V)", "Tensión: L3 (V)", "Tensión: L1 - L2 (V)", 
    "Tensión: L2 - L3 (V)", "Tensión: L3 - L1 (V)", "timestamp", "Corriente: L1 (A)", "Corriente: L2 (A)", 
    "Corriente: L3 (A)", "Factor de potencia: L1 +", "Factor de potencia: L2 +", 
    "Factor de potencia: L3 +", "Flicker (Pst): L1 (Pst)", "Flicker (Pst): L2 (Pst)", 
    "Flicker (Pst): L3 (Pst)", "Distorsión armónica: VL1 (%V THD)", 
    "Distorsión armónica: VL2 (%V THD)", "Distorsión armónica: VL3 (%V THD)", 
    "Distorsión armónica: IL1 (%I THD)", "Distorsión armónica: IL2 (%I THD)", 
    "Distorsión armónica: IL3 (%I THD)", "Armónicos IL1: Armónico 2 (%IL1)", 
    "Armónicos IL1: Armónico 3 (%IL1)", "Armónicos IL1: Armónico 4 (%IL1)", 
    "Armónicos IL1: Armónico 5 (%IL1)", "Armónicos IL1: Armónico 6 (%IL1)", 
    "Armónicos IL1: Armónico 7 (%IL1)", "Armónicos IL1: Armónico 8 (%IL1)", 
    "Armónicos IL1: Armónico 9 (%IL1)", "Armónicos IL1: Armónico 10 (%IL1)", 
    "Armónicos IL1: Armónico 11 (%IL1)", "Armónicos IL1: Armónico 12 (%IL1)", 
    "Armónicos IL1: Armónico 13 (%IL1)", "Armónicos IL1: Armónico 14 (%IL1)", 
    "Armónicos IL1: Armónico 15 (%IL1)", "Armónicos IL2: Armónico 2 (%IL2)", 
    "Armónicos IL2: Armónico 3 (%IL2)", "Armónicos IL2: Armónico 4 (%IL2)", 
    "Armónicos IL2: Armónico 5 (%IL2)", "Armónicos IL2: Armónico 6 (%IL2)", 
    "Armónicos IL2: Armónico 7 (%IL2)", "Armónicos IL2: Armónico 8 (%IL2)", 
    "Armónicos IL2: Armónico 9 (%IL2)", "Armónicos IL2: Armónico 10 (%IL2)", 
    "Armónicos IL2: Armónico 11 (%IL2)", "Armónicos IL2: Armónico 12 (%IL2)", 
    "Armónicos IL2: Armónico 13 (%IL2)", "Armónicos IL2: Armónico 14 (%IL2)", 
    "Armónicos IL2: Armónico 15 (%IL2)", "Armónicos VL1: Armónico 2 (%VL1)", 
    "Armónicos VL1: Armónico 3 (%VL1)", "Armónicos VL1: Armónico 4 (%VL1)", 
    "Armónicos VL1: Armónico 5 (%VL1)", "Armónicos VL1: Armónico 6 (%VL1)", 
    "Armónicos VL1: Armónico 7 (%VL1)", "Armónicos VL1: Armónico 8 (%VL1)", 
    "Armónicos VL1: Armónico 9 (%VL1)", "Armónicos VL1: Armónico 10 (%VL1)", 
    "Armónicos VL1: Armónico 11 (%VL1)", "Armónicos VL1: Armónico 12 (%VL1)", 
    "Armónicos VL1: Armónico 13 (%VL1)", "Armónicos VL1: Armónico 14 (%VL1)", 
    "Armónicos VL1: Armónico 15 (%VL1)", "Armónicos VL2: Armónico 2 (%VL2)", 
    "Armónicos VL2: Armónico 3 (%VL2)", "Armónicos VL2: Armónico 4 (%VL2)", 
    "Armónicos VL2: Armónico 5 (%VL2)", "Armónicos VL2: Armónico 6 (%VL2)", 
    "Armónicos VL2: Armónico 7 (%VL2)", "Armónicos VL2: Armónico 8 (%VL2)", 
    "Armónicos VL2: Armónico 9 (%VL2)", "Armónicos VL2: Armónico 10 (%VL2)", 
    "Armónicos VL2: Armónico 11 (%VL2)", "Armónicos VL2: Armónico 12 (%VL2)", 
    "Armónicos VL2: Armónico 13 (%VL2)", "Armónicos VL2: Armónico 14 (%VL2)", 
    "Armónicos VL2: Armónico 15 (%VL2)", "Armónicos VL3: Armónico 2 (%VL3)", 
    "Armónicos VL3: Armónico 3 (%VL3)", "Armónicos VL3: Armónico 4 (%VL3)", 
    "Armónicos VL3: Armónico 5 (%VL3)", "Armónicos VL3: Armónico 6 (%VL3)", 
    "Armónicos VL3: Armónico 7 (%VL3)", "Armónicos VL3: Armónico 8 (%VL3)", 
    "Armónicos VL3: Armónico 9 (%VL3)", "Armónicos VL3: Armónico 10 (%VL3)", 
    "Armónicos VL3: Armónico 11 (%VL3)", "Armónicos VL3: Armónico 12 (%VL3)", 
    "Armónicos VL3: Armónico 13 (%VL3)", "Armónicos VL3: Armónico 14 (%VL3)", 
    "Armónicos VL3: Armónico 15 (%VL3)"
]

# Filtrar solo las columnas especificadas
filtered_data = data[columns_to_keep]

# Guardar el archivo filtrado en la misma ubicación con un nuevo nombre
output_path = '/home/elicoubuntu/Desktop/Industrial_Insigths/Prognosis/filtered_consolidated_data.xlsx'
filtered_data.to_excel(output_path, index=False)

print(f"Archivo guardado como {output_path}")



Archivo guardado como /home/elicoubuntu/Desktop/Industrial_Insigths/Prognosis/filtered_consolidated_data.xlsx


In [92]:
filtered_data.head()

,Tensión: L1 (V),Tensión: L2 (V),Tensión: L3 (V),Tensión: L1 - L2 (V),Tensión: L2 - L3 (V),Tensión: L3 - L1 (V),timestamp,Corriente: L1 (A),Corriente: L2 (A),Corriente: L3 (A),...,Armónicos VL3: Armónico 6 (%VL3),Armónicos VL3: Armónico 7 (%VL3),Armónicos VL3: Armónico 8 (%VL3),Armónicos VL3: Armónico 9 (%VL3),Armónicos VL3: Armónico 10 (%VL3),Armónicos VL3: Armónico 11 (%VL3),Armónicos VL3: Armónico 12 (%VL3),Armónicos VL3: Armónico 13 (%VL3),Armónicos VL3: Armónico 14 (%VL3),Armónicos VL3: Armónico 15 (%VL3)
0,264.42,268.32,266.73,462.09,464.87,457.94,2013-05-04 16:07:00,477.28,566.28,564.68,...,0,1.0,0,0.0,0,0.2,0,0.0,0,0
1,264.73,268.59,266.98,462.57,465.30,458.48,2013-05-04 16:08:00,454.16,541.36,538.44,...,0,1.0,0,0.0,0,0.2,0,0.0,0,0
2,264.78,268.65,267.08,462.63,465.47,458.60,2013-05-04 16:09:00,449.76,537.60,534.96,...,0,1.0,0,0.0,0,0.2,0,0.0,0,0
3,264.77,268.60,267.08,462.60,465.41,458.60,2013-05-04 16:10:00,459.40,548.80,544.68,...,0,1.1,0,0.0,0,0.2,0,0.0,0,0
4,264.85,268.65,267.06,462.73,465.40,458.65,2013-05-04 16:11:00,463.12,551.08,547.12,...,0,1.2,0,0.0,0,0.1,0,0.0,0,0


1. Análisis preliminar y preparación
Objetivo: Configurar el entorno de trabajo y realizar una exploración inicial de los datos.
Exploración inicial:

Cargar y explorar el dataset (filtered_consolidated_data.xlsx).
Identificar variables clave para el análisis (tensiones, corrientes, armónicos, timestamp, etc.).
Detectar y tratar valores faltantes o inconsistencias.
Limpieza de datos:

Eliminar columnas constantes o irrelevantes.
Validar el formato de las columnas temporales y convertir timestamp a formato datetime.
Normalización:

Escalar las variables numéricas entre 0 y 1 para uniformizar los rangos.
Definir métricas de evaluación:

Seleccionar métricas para evaluar el desempeño del algoritmo en términos de detección de anomalías y tendencias (e.g., precisión, recall, F1).
2. Implementación de la línea base (clustering inicial)
Objetivo: Identificar patrones normales del sistema para establecer un modelo inicial.
Análisis de agrupamiento:

Aplicar K-Means para identificar comportamientos normales y patrones dentro del dataset.
Utilizar el método del codo para determinar el número óptimo de clústeres.
Asignación de clústeres:

Etiquetar cada registro con su clúster correspondiente.
Calcular la distancia de cada registro al centroide de su clúster como indicador de anomalía.
Validación del clustering:

3. Implementación de detección de anomalías
Objetivo: Identificar registros que se desvíen significativamente del comportamiento normal.
Umbral basado en distancias:

Calcular percentiles (e.g., 95%) para establecer un umbral adaptativo de anomalías basado en la distancia al centroide.
Técnicas de detección:

Comparar métodos como Isolation Forest, DBSCAN y basados en percentiles.
Etiquetar registros como normales o anómalos.
Validación:

Verificar la proporción de anomalías detectadas y su coherencia con el comportamiento esperado.
4. Análisis de tendencias
Objetivo: Detectar cambios significativos a lo largo del tiempo en las variables monitoreadas.
Modelo de series temporales:

Aplicar Prophet o ARIMA para identificar tendencias, estacionalidades y variaciones en variables clave.
Evaluar la estabilidad de la línea base con datos históricos.
Integración con anomalías:

Combinar tendencias detectadas con anomalías para obtener un análisis contextualizado.
Visualización de resultados:

Generar gráficos de variables clave con superposición de anomalías y tendencias.
5. Evaluación frente a límites establecidos
Objetivo: Incorporar límites críticos que definan condiciones de falla potencial.
Definición de límites:

Configurar límites fijos o dinámicos basados en estándares del sistema o cálculos históricos.
Comparación con límites:

Evaluar si los registros exceden los límites y generar alertas si es necesario.
Validación:

Asegurar que las alertas reflejen eventos reales y sean interpretables.
6. Cálculo de probabilidad de falla
Objetivo: Generar una métrica interpretativa para cada registro basado en anomalías, tendencias y límites.
Cálculo inicial:

Definir la probabilidad como función ponderada de:
Distancia al centroide (anomalías).
Cambios en tendencias.
Desviaciones frente a límites.
Ajustes dinámicos:

Permitir que los usuarios ajusten parámetros (e.g., pesos) desde una interfaz.
7. Desarrollo de la interfaz en el entorno de pruebas
Objetivo: Simular la visualización y parametrización del algoritmo.
Visualización inicial:

Mostrar gráficos de:
Comportamiento del sistema en el tiempo.
Distribución de clústeres y anomalías.
Parametrización:

Permitir a los usuarios modificar límites y pesos de las métricas en tiempo real.
8. Validación en el entorno de pruebas
Objetivo: Verificar que el algoritmo cumple con los objetivos antes de escalar a producción.
Pruebas de rendimiento:

Evaluar la capacidad del algoritmo con datasets más grandes.
Medir tiempos de procesamiento y memoria.
Pruebas de precisión:

Confirmar la coherencia de las detecciones de fallas.
9. Escalado a producción con MLOps
Objetivo: Implementar el algoritmo en un entorno escalable.
Despliegue del modelo:

Contenerizar los componentes del sistema usando Docker.
Implementar un pipeline de datos para ingestión en tiempo real.
Monitorización:

Configurar métricas de desempeño y alertas para el sistema.
Mantenimiento:

Actualizar modelos periódicamente con nuevos datos.
10. Documentación y capacitación
Objetivo: Facilitar el uso y mantenimiento del sistema.
Documentación técnica:
Detallar cada componente del sistema y su propósito.
Capacitación:
Entrenar a los usuarios sobre cómo interpretar las salidas y ajustar parámetros.


Fitrar las variables constantes: 

Qué son las columnas constantes?
Son variables en el dataset que tienen el mismo valor para todos los registros. En este caso, identificamos 35 columnas constantes, principalmente relacionadas con armónicos.
¿Por qué eliminarlas?
Sin aporte informativo:

Estas columnas no contribuyen al análisis porque no varían entre los datos. Por ejemplo, si un armónico es siempre 0%, no aporta información útil para detectar anomalías o cambios en tendencias.
Reducción de dimensionalidad:

Al eliminar estas columnas, reducimos la complejidad del modelo y el tiempo de procesamiento sin perder información relevante.
Optimización de recursos:

Menos columnas implican menos memoria utilizada y procesamiento más eficiente, crucial en un sistema en tiempo real.

In [1]:
import pandas as pd

# Cargar el archivo Excel
file_path = "/home/elicoubuntu/Desktop/Industrial_Insigths/Prognosis/filtered_consolidated_data.xlsx"
df = pd.read_excel(file_path)

# Identificar columnas constantes
constant_columns = [col for col in df.columns if df[col].nunique() == 1]

# Eliminar columnas constantes
df_cleaned = df.drop(columns=constant_columns)

# Guardar el dataset limpio en un nuevo archivo
cleaned_file_path = "/home/elicoubuntu/Desktop/Industrial_Insigths/Prognosis/filtered_consolidated_data_cleaned.xlsx"
df_cleaned.to_excel(cleaned_file_path, index=False)

print(f"Se eliminaron {len(constant_columns)} columnas constantes.")
print(f"Archivo limpio guardado en: {cleaned_file_path}")


Se eliminaron 35 columnas constantes.
Archivo limpio guardado en: /home/elicoubuntu/Desktop/Industrial_Insigths/Prognosis/filtered_consolidated_data_cleaned.xlsx
